In [43]:
from collections import defaultdict

dupe_dict = [{'id': 1, 'parent_id': 22, 'name': "foo"}, # ok
             {'id': 2, 'parent_id': 22, 'name': "foo"}, # not ok, same name and parent_id, different feature id
             {'id': 5, 'parent_id': 25, 'name': "foo"}, # ok, same name, different id, different parent_id
             {'id': 3, 'parent_id': 23, 'name': "bar"}, # ok
             {'id': 3, 'parent_id': 23, 'name': "bar"}] # ok, same name, but also same id, like (es_ES and pt_BR)

def read_names(dupe_dict):
    '''Look for names that are duplicates and have the same parent_id, but belong to different features.
       Essentially this creates a "name" index.  Each unique name in a role is indexed and has the parent_id
       and feature_ids attached to it'''
    name_holder = defaultdict(set)
    for name_entry in dupe_dict:
        id = name_entry['id']
        name = name_entry['name']
        parent_id = name_entry['parent_id']
        if name not in name_holder:
            name_holder[name] = {parent_id: set()}
            name_holder[name][parent_id].add(id)
        if parent_id not in name_holder[name]:
            name_holder[name][parent_id] = set()
            name_holder[name][parent_id].add(id)
        else:
            name_holder[name][parent_id].add(id)
    return name_holder

def dupe_check(parsed_names):
    '''Checks to see if any name is used more than once for different features with the same parent_id.'''
    for name in parsed_names:
        for parent_id in parsed_names[name]:
            id_count = len(parsed_names[name][parent_id])
            print name, parent_id, id_count
            
        
parsed_names = read_names(dupe_dict)
dupes = dupe_check(parsed_names)

foo 25 1
foo 22 2
bar 23 1


In [ ]:
import unittest
from fdb_test_base import FDBTestBase
from teamcity import is_running_under_teamcity
from teamcity.unittestpy import TeamcityTestRunner

class TestDupeNames(FDBTestBase):

    def testCountryNames(self):
        map_codes = ['0','1','2','3']
        test_query = '''SELECT "CountrySynonyms"."ParentID" as "ID",
                               "CountrySynonyms"."Name",
                               "CountrySynonyms"."MapCode",
                               "Country"."ParentID"
                               FROM "CountrySynonyms"
                               JOIN "Country" ON "CountrySynonyms"."ParentID" = "Country"."ID" 
                               WHERE "CountrySynonyms"."MapCode" = '{}' '''
        errors = []
        for map_code in map_codes:
            results = self.fdb.execute_dict_query(test_query.format(map_code))
            print results
        #self.assertTrue(errors == False, errors)

if __name__ == '__main__':
    if is_running_under_teamcity():
        runner = TeamcityTestRunner()
    else:
        runner = unittest.TextTestRunner()
    unittest.main(testRunner=runner)
                                                                   